# CNN on MNIST dataset

Let's start by importing numpy and setting a seed for the computer's pseudorandom number generator. This allows us to reproduce the results from our script:

In [2]:
import numpy as np
np.random.seed(123)  # for reproducibility

Next, we'll import the Sequential model type from Keras. This is simply a linear stack of neural network layers, and it's perfect for the type of feed-forward CNN we're building.

In [3]:
from keras.models import Sequential

Using TensorFlow backend.


Next, we import the "core" layers from Keras. These are the layers that are used in almost any neural network:

In [4]:
from keras.layers import Dense, Dropout, Activation, Flatten

# Dropout layer:

Dropout is a technique that randomly ignoring nodes. It is useful because it prevents inter-dependencies from emerging between nodes (I.e. nodes do not learn functions which rely on input values from another node), this allows the network to learn more a more robust relationship. Implementing dropout has much the same affect as taking the average from a committee of networks, however the cost is significantly less in both time and storage required.

# Dense layer:

It is the fully connected layer in our neural network.

# Activation layer:

After each conv layer, it is convention to apply a nonlinear layer (or activation layer) immediately afterward.The purpose of this layer is to introduce nonlinearity to a system that basically has just been computing linear operations during the conv layers (just element wise multiplications and summations). It also helps to alleviate the vanishing gradient problem, which is the issue where the lower layers of the network train very slowly because the gradient decreases exponentially through the layers

# Flatten layer:


We need to convert the output of the convolutional part of the CNN into a 1D feature vector, to be used by the ANN part of it. This operation is called flattening. It gets the output of the convolutional layers, flattens all its structure to create a single long feature vector to be used by the dense layer for the final classification.

Now we'll import the CNN layers from Keras. These are the convolutional layers that will help us efficiently train on image data. We will also import some utilities. This will help us transform our data later:

In [5]:
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
import matplotlib.pyplot as plt

# Network Parameters:

In [6]:
batch_size = 128 #The batch size, representing the number of 
                 #training examples being used simultaneously during a single iteration of the gradient 
                 #descent algorithm
nb_classes = 10
nb_epoch = 12

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
nb_filters = 32
# size of pooling area for max pooling
pool_size = (2, 2)
# convolution kernel size
kernel_size = (3, 3)

# Loading the data from MNIST:

In [7]:
from keras.datasets import mnist
 
# Load pre-shuffled MNIST data into train and test sets

(X_train, y_train), (X_test, y_test) = mnist.load_data()

# MNIST:

The MNIST database (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used for training various image processing systems.

# Preprocess input data for Keras.

Our MNIST images only have a depth of 1, but we must explicitly declare that.
In other words, we want to transform our dataset from having shape (n, width, height) to (n, depth, width, height).

In [8]:
X_train = X_train.reshape(X_train.shape[0], img_cols, img_rows, 1)
X_test = X_test.reshape(X_test.shape[0], img_cols, img_rows, 1)
input_shape=(img_cols, img_rows, 1)

The final preprocessing step for the input data is to convert our data type to float32 and normalize our data values to the range [0, 1].

In [9]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

### Information about the data:

In [10]:
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


The y_train and y_test data are not split into 10 distinct class labels, but rather are represented as a single array with the class values. We can fix this easily by converting 1-dimensional class arrays to 10-dimensional class matrices.

In [11]:
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

# Building the CNN:

We start by declaring a sequential model:

In [12]:
model = Sequential()

Next we declare the input layers:

In [13]:
model.add(Convolution2D(32, kernel_size=(3, 3),
                 activation='relu',
input_shape=input_shape))

The input shape parameter should be the shape of 1 sample. In this case, it's the same (1, 28, 28) that corresponds to  the (depth, width, height) of each digit image.

But what do the first 3 parameters represent? They correspond to the number of convolution filters to use, the number of rows in each convolution kernel, and the number of columns in each convolution kernel, respectively. 

We can add more layers to our model.

In [14]:
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

# Complie the model

model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
  


Show a summary of the model parameters.

In [15]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 4608)              0         
__________

# Fit model on training data:

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 496s 8ms/step - loss: 0.3743 - acc: 0.8852 - val_loss: 0.0890 - val_acc: 0.9726
Epoch 2/12
60000/60000 [==============================] - 480s 8ms/step - loss: 0.1341 - acc: 0.9599 - val_loss: 0.0634 - val_acc: 0.9793
Epoch 3/12
60000/60000 [==============================] - 479s 8ms/step - loss: 0.1059 - acc: 0.9691 - val_loss: 0.0484 - val_acc: 0.9840
Epoch 4/12
60000/60000 [==============================] - 992s 17ms/step - loss: 0.0901 - acc: 0.9727 - val_loss: 0.0451 - val_acc: 0.9861
Epoch 5/12
23040/60000 [==========>...................] - ETA: 5:05 - loss: 0.0786 - acc: 0.9769

# Evaluate the model:

In [1]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

NameError: name 'model' is not defined

# Plot the graphs for better understanding:

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()